In [1]:
import os
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import jaccard_score

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.python.keras.models import *

from tensorflow.python.keras.utils.generic_utils import get_custom_objects

os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
epsilon = K.epsilon()
gamma = 0
alpha = 0.6
beta = 0.6

  
def recall(y_true, y_pred):

    y_true_yn = K.round(K.clip(y_true, 0, 1))
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    count_true_positive = K.sum(y_true_yn * y_pred_yn)     
    count_true_positive_false_negative = K.sum(y_true_yn)
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    return recall


def precision(y_true, y_pred):

    y_pred_yn = K.round(K.clip(y_pred, 0, 1))
    y_true_yn = K.round(K.clip(y_true, 0, 1))
    count_true_positive = K.sum(y_true_yn * y_pred_yn) 
    count_true_positive_false_positive = K.sum(y_pred_yn)
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    return precision


def f1score(y_true, y_pred):
    _recall = recall(y_true, y_pred)
    _precision = precision(y_true, y_pred)
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())

    return _f1score

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


def balanced_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    BL = alpha * CE
    
    return K.sum(BL, axis=1)


def focal_loss(y_true, y_pred):
    pt = y_pred * y_true + (1-y_pred) * (1-y_true)
    pt = K.clip(pt, epsilon, 1-epsilon)
    CE = -K.log(pt)
    FL = alpha * K.pow(1-pt, gamma) * CE
    
    return K.sum(FL, axis=1)


def dice_coef(y_true, y_pred, smooth=0.001):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def cus_loss(y_true, y_pred):
    
    return (1 - beta) * focal_loss(y_true, y_pred) + beta * dice_coef_loss(y_true, y_pred)


get_custom_objects().update({
    
    'cus_loss': cus_loss,
    'iou_coef' : iou_coef,
    'f1score' : f1score,
    'precision' : precision,
    'recall' : recall,
    'balanced_loss' : balanced_loss,
    'focal_loss' : focal_loss,
    'dice_coef' : dice_coef,
    'dice_coef_loss' : dice_coef_loss,
    'cus_loss' : cus_loss,
        
})

In [3]:
train_a2c_image = np.load('../../data/dataset/train/train_A2C_image.npy').astype(np.float32)
train_a2c_label = np.load('../../data/dataset/train/train_A2C_label.npy').astype(np.float32)

train_a4c_image = np.load('../../data/dataset/train/train_A4C_image.npy').astype(np.float32)
train_a4c_label = np.load('../../data/dataset/train/train_A4C_label.npy').astype(np.float32)


test_a2c_image = np.load('../../data/dataset/test/test_A2C_image.npy').astype(np.float32)
test_a2c_label = np.load('../../data/dataset/test/test_A2C_label.npy').astype(np.float32)

test_a4c_image = np.load('../../data/dataset/test/test_A4C_image.npy').astype(np.float32)
test_a4c_label = np.load('../../data/dataset/test/test_A4C_label.npy').astype(np.float32)

In [4]:
print(train_a2c_image.min(), train_a2c_image.max())
print(train_a2c_label.min(), train_a2c_label.max())


print(train_a4c_image.min(), train_a4c_image.max())
print(train_a4c_label.min(), train_a4c_label.max())


print(test_a2c_image.min(), test_a2c_image.max())
print(test_a2c_label.min(), test_a2c_label.max())


print(test_a4c_image.min(), test_a4c_image.max())
print(test_a4c_label.min(), test_a4c_label.max())

0.0 255.0
0.0 1.0
0.0 255.0
0.0 1.0
0.0 255.0
0.0 1.0
0.0 255.0
0.0 1.0


In [5]:
train_a2c_image = train_a2c_image / 255
train_a4c_image = train_a4c_image / 255

test_a2c_image = test_a2c_image / 255
test_a4c_image = test_a4c_image / 255

In [6]:
print(train_a2c_image.min(), train_a2c_image.max())
print(train_a2c_label.min(), train_a2c_label.max())


print(train_a4c_image.min(), train_a4c_image.max())
print(train_a4c_label.min(), train_a4c_label.max())


print(test_a2c_image.min(), test_a2c_image.max())
print(test_a2c_label.min(), test_a2c_label.max())


print(test_a4c_image.min(), test_a4c_image.max())
print(test_a4c_label.min(), test_a4c_label.max())

0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [7]:
print(train_a2c_image.shape)
print(train_a2c_label.shape)


print(train_a4c_image.shape)
print(train_a4c_label.shape)


print(test_a2c_image.shape)
print(test_a2c_label.shape)


print(test_a4c_image.shape)
print(test_a4c_label.shape)

(800, 512, 512, 1)
(800, 512, 512, 1)
(800, 512, 512, 1)
(800, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)
(100, 512, 512, 1)


In [8]:
all_train_image = np.zeros((800,512,512,1,2))
all_train_label = np.zeros((800,512,512,1,2))

test_image = np.zeros((100,512,512,1,2))
test_label = np.zeros((100,512,512,1,2))

In [9]:
all_train_image[:,:,:,:,0] = train_a2c_image
all_train_image[:,:,:,:,1] = train_a4c_image

all_train_label[:,:,:,:,0] = train_a2c_label
all_train_label[:,:,:,:,1] = train_a4c_label

test_image[:,:,:,:,0] = test_a2c_image
test_image[:,:,:,:,1] = test_a4c_image

test_label[:,:,:,:,0] = test_a2c_label
test_label[:,:,:,:,1] = test_a4c_label

In [10]:
train_image_fold_1 = all_train_image[:200]
train_image_fold_2 = all_train_image[200:400]
train_image_fold_3 = all_train_image[400:600]
train_image_fold_4 = all_train_image[600:]


train_label_fold_1 = all_train_label[:200]
train_label_fold_2 = all_train_label[200:400]
train_label_fold_3 = all_train_label[400:600]
train_label_fold_4 = all_train_label[600:]

In [11]:
# # CV1
# train_image = np.concatenate((train_image_fold_1, train_image_fold_2, train_image_fold_3), axis =0)
# train_label = np.concatenate((train_label_fold_1, train_label_fold_2, train_label_fold_3), axis =0)

# validation_image = train_image_fold_4
# validation_label = train_label_fold_4

In [12]:
# CV2
train_image = np.concatenate((train_image_fold_1, train_image_fold_2, train_image_fold_4), axis =0)
train_label = np.concatenate((train_label_fold_1, train_label_fold_2, train_label_fold_4), axis =0)

validation_image = train_image_fold_3
validation_label = train_label_fold_3

In [13]:
# # CV3
# train_image = np.concatenate((train_image_fold_1, train_image_fold_3, train_image_fold_4), axis =0)
# train_label = np.concatenate((train_label_fold_1, train_label_fold_3, train_label_fold_4), axis =0)

# validation_image = train_image_fold_2
# validation_label = train_label_fold_2

In [14]:
# # CV4
# train_image = np.concatenate((train_image_fold_2, train_image_fold_3, train_image_fold_4), axis =0)
# train_label = np.concatenate((train_label_fold_2, train_label_fold_3, train_label_fold_4), axis =0)

# validation_image = train_image_fold_1
# validation_label = train_label_fold_1

In [15]:
print(train_image.shape)
print(train_label.shape)

print(validation_image.shape)
print(validation_label.shape)

(600, 512, 512, 1, 2)
(600, 512, 512, 1, 2)
(200, 512, 512, 1, 2)
(200, 512, 512, 1, 2)


In [16]:
def Conv2D_block(input_layer, out_n_filters, kernel_size=[3,3], stride=[1,1], padding='same'):
    
    layer = input_layer
    
    for i in range(2):
        
        layer = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, kernel_initializer = 'he_normal')(layer)
        layer = BatchNormalization()(layer)
        layer = Activation('relu')(layer)        
        
    out_layer = layer
    
    return out_layer
    

def Up_and_Concate(down_layer, layer):
    
    input_channel = down_layer.get_shape().as_list()[3]
    output_channel = input_channel // 2
    
    up = UpSampling2D(size = (2,2))(down_layer) 

    concate = concatenate([up, layer])
    return concate

def attention_block_2d(x, g, inter_channel):


    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1])(x)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1])(g)

    f = Activation('relu')(add([theta_x, phi_g]))


    psi_f = Conv2D(1, [1, 1], strides=[1, 1])(f)

    rate = Activation('sigmoid')(psi_f)


    att_x = multiply([x, rate])

    return att_x


def attention_up_and_concate(down_layer, layer):
    in_channel = down_layer.get_shape().as_list()[3]

    up = UpSampling2D((2, 2))(down_layer)

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4)

    concate = concatenate([up, layer])
    
    return concate

In [17]:
def AU_Net_2D(input_shape):
        
    inputs = Input(shape=input_shape)
    x = inputs
    depth = 4
    features = 32
    down_layer = []
    supervision_layer = []
    
    for i in range(depth):
        
        x = Conv2D_block(x, features)
        down_layer.append(x)
        x = MaxPooling2D(pool_size=[2, 2], strides=[2, 2])(x)

        features = features * 2
        
    x = Conv2D_block(x, features)
    
    for i in reversed(range(depth)):

        features = features // 2
        
        x = attention_up_and_concate(x, down_layer[i])
        x = Conv2D_block(x, features)
        supervision_layer.append(x)
    
    

    output_2 = UpSampling2D((4, 4))(supervision_layer[1])
    output_4 = UpSampling2D((1, 1))(supervision_layer[3])
    

    model = Model(inputs = inputs, outputs = [output_2, output_4])

    
    return model

In [18]:
base_model = AU_Net_2D((512,512,1))
base_model.summary()

W1207 15:13:44.722079 139067977648272 deprecation.py:506] From /opt/anaconda3/envs/powerai_162/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 512, 512, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [19]:
def Multi_View_AU_Net():
    
    inputs = Input(shape=(512,512,1,2), name='input')
    
    a2c_view = inputs[:, :, :, :, 0]
    a4c_view = inputs[:, :, :, :, 1]
    
    base_model = AU_Net_2D(input_shape = (512,512,1))
    
    a2c_view_output_2, a2c_view_output_4 = base_model(a2c_view)
    a4c_view_output_2, a4c_view_output_4 = base_model(a4c_view)
    
    
    output_2 = Concatenate()([a2c_view_output_2, a4c_view_output_2])
    output_4 = Concatenate()([a2c_view_output_4, a4c_view_output_4])

    
    fn_output_2 = Dense(32)(output_2)
    fn_output_2 = Dropout(0.25)(fn_output_2)
    fn_output_2 = Dense(2)(fn_output_2)
    
    
    fn_output_4 = Dense(32)(output_4)
    fn_output_4 = Dropout(0.25)(fn_output_4)
    fn_output_4 = Dense(2)(fn_output_4)
    
    

    fn_output_2 = K.expand_dims(fn_output_2, 3)
    fn_output_4 = K.expand_dims(fn_output_4, 3)
    

    fn_output_2 = Activation('sigmoid', name='out_2')(fn_output_2)
    fn_output_4 = Activation('sigmoid', name='out_4')(fn_output_4)


    MVAU_Net = keras.Model(inputs = inputs, outputs = [fn_output_2, fn_output_4], name='MV_AU_Net')

    
    return MVAU_Net

In [20]:
MVAU_Net = Multi_View_AU_Net()
MVAU_Net.summary(line_length=150)

W1207 15:13:47.408190 139067977648272 backend.py:548] OMP_NUM_THREADS is no longer used by the default Keras config. To configure the number of threads, use tf.config.threading APIs.


Model: "MV_AU_Net"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input (InputLayer)                               [(None, 512, 512, 1, 2)]         0                                                                   
______________________________________________________________________________________________________________________________________________________
tf_op_layer_strided_slice (TensorFlowOpLayer)    [(None, 512, 512, 1)]            0                 input[0][0]                                       
______________________________________________________________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (TensorFlowOpLayer)  [(None, 512, 512, 1)]     

In [21]:
losses ={'out_2':dice_coef_loss,
         'out_4':dice_coef_loss}

MVAU_Net.compile(optimizer=Adam(lr=0.001), loss = losses)

In [22]:
# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.3, patience=5, verbose=1, min_delta=1e-8)
earlystopper = EarlyStopping(monitor='loss',patience=30, verbose=1)
model_checkpoint = ModelCheckpoint('../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5', monitor='loss', verbose=1, save_best_only=True)

callbacks_list = [model_checkpoint, earlystopper]

In [23]:
print('Fitting model...')
print('-'*200)
hist = MVAU_Net.fit(train_image, [train_label, train_label], batch_size=4, epochs=100, verbose=1, callbacks=callbacks_list)

Fitting model...
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Train on 600 samples
Epoch 1/100
596/600 [============================>.] - ETA: 0s - loss: 0.4904 - out_2_loss: 0.2206 - out_4_loss: 0.2698
Epoch 00001: loss improved from inf to 0.48896, saving model to ../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5
600/600 [==============================] - 66s 110ms/sample - loss: 0.4890 - out_2_loss: 0.2200 - out_4_loss: 0.2689
Epoch 2/100
596/600 [============================>.] - ETA: 0s - loss: 0.2128 - out_2_loss: 0.1060 - out_4_loss: 0.1068
Epoch 00002: loss improved from 0.48896 to 0.21243, saving model to ../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5
600/600 [==============================] - 58s 97ms/sample - loss: 0.2124 - out_2_loss: 0.1058 - out_4_loss: 0.1066
Epoch 3/100
596/600 [=====================

Epoch 24/100
596/600 [============================>.] - ETA: 0s - loss: 0.0823 - out_2_loss: 0.0437 - out_4_loss: 0.0386
Epoch 00024: loss improved from 0.08364 to 0.08214, saving model to ../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5
600/600 [==============================] - 60s 100ms/sample - loss: 0.0821 - out_2_loss: 0.0437 - out_4_loss: 0.0385
Epoch 25/100
596/600 [============================>.] - ETA: 0s - loss: 0.0837 - out_2_loss: 0.0443 - out_4_loss: 0.0394
Epoch 00025: loss did not improve from 0.08214
600/600 [==============================] - 58s 97ms/sample - loss: 0.0837 - out_2_loss: 0.0443 - out_4_loss: 0.0394
Epoch 26/100
596/600 [============================>.] - ETA: 0s - loss: 0.0790 - out_2_loss: 0.0421 - out_4_loss: 0.0369
Epoch 00026: loss improved from 0.08214 to 0.07890, saving model to ../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5
600/600 [==============================] - 59s 99ms/sample - loss: 0.0789 - out_2_loss: 0.0421 - out_4_loss

600/600 [==============================] - 59s 98ms/sample - loss: 0.0354 - out_2_loss: 0.0226 - out_4_loss: 0.0127
Epoch 73/100
596/600 [============================>.] - ETA: 0s - loss: 0.0349 - out_2_loss: 0.0224 - out_4_loss: 0.0125
Epoch 00073: loss improved from 0.03537 to 0.03492, saving model to ../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5
600/600 [==============================] - 61s 101ms/sample - loss: 0.0349 - out_2_loss: 0.0224 - out_4_loss: 0.0125
Epoch 74/100
596/600 [============================>.] - ETA: 0s - loss: 0.0348 - out_2_loss: 0.0224 - out_4_loss: 0.0123
Epoch 00074: loss improved from 0.03492 to 0.03479, saving model to ../../result/model_save/CV/A2C_A4C_Deep_MVAU_Net_CV_2.h5
600/600 [==============================] - 60s 99ms/sample - loss: 0.0348 - out_2_loss: 0.0224 - out_4_loss: 0.0123
Epoch 75/100
596/600 [============================>.] - ETA: 0s - loss: 0.0346 - out_2_loss: 0.0224 - out_4_loss: 0.0123
Epoch 00075: loss improved from 0.0347

In [31]:
a2c_a4c_pred_2, a2c_a4c_pred_4 = MVAU_Net.predict(validation_image, verbose=1)
a2c_a4c_pred_4 = np.array(a2c_a4c_pred_4 > 0.5)

200/200 [==============================] - 8s 39ms/sample


In [30]:
_, test_prediction = MVAU_Net.predict(test_image, verbose=1)
test_prediction = np.array(test_prediction > 0.5)

100/100 [==============================] - 4s 39ms/sample


In [32]:
np.save('../../result/CV/CV_2_valid_prediction.npy', a2c_a4c_pred_4)
np.save('../../result/CV/CV_2_test_prediction.npy', test_prediction)

In [25]:
a2c_a4c_pred_2 = np.array(a2c_a4c_pred_2 > 0.5)
a2c_a4c_pred_4 = np.array(a2c_a4c_pred_4 > 0.5)

recall_list = []
pricision_list = []
acc_list = []
dice_list = []
jaccard_list = []


for i in range(test_label.shape[0]):
    
    seg1 = a2c_a4c_pred_4[i,:,:,0,0]
    gt1 = validation_label[i,:,:,0,0]

    seg1_n = seg1 == 0
    seg1_t = seg1 == 1

    gt1_n = gt1 == 0
    gt1_t = gt1 == 1


    tp = np.sum(seg1_t&gt1_t)

    fp = np.sum(seg1_t&gt1_n)

    tn = np.sum(seg1_n&gt1_n)

    fn = np.sum(gt1_t&seg1_n)


    recall = tp / (tp + fn)
    recall_list.append(recall)

#     print('Sensitivity : ', sensitivity)


    pricision = tp / (tp+fp)
    pricision_list.append(pricision)

#     print('Specificity : ', specificity)


    acc = (tp + tn) / (tp + tn + fp + fn)
    acc_list.append(acc)
#     print('Accuracy : ', acc)

    dice = (2*tp) / (2*tp + fp + fn)
    dice_list.append(dice)
#     print('DSC : ', dice)
    
    jaccard_list.append(jaccard_score(gt1, seg1, average='micro'))
    
    
dice_list_1 = np.array(dice_list)
recall_list_1 = np.array(recall_list)
pricision_list_1 = np.array(pricision_list)
jaccard_list_1 = np.array(jaccard_list)

dice_list_1 = dice_list_1[~np.isnan(dice_list_1)]
recall_list_1 = recall_list_1[~np.isnan(recall_list_1)]
pricision_list_1 = pricision_list_1[~np.isnan(pricision_list_1)]
jaccard_list_1 = jaccard_list_1[~np.isnan(jaccard_list_1)]

print('Recall    : ', round(np.mean(recall_list_1), 3), '±', round(np.std(recall_list_1), 3))
print('Precision : ', round( np.mean(pricision_list_1), 3), '±', round( np.std(pricision_list_1), 3))
print('Accuracy  : ', round(np.mean(acc_list), 3), '±', round( np.std(acc_list), 3))
print('DSC       : ', round(np.mean(dice_list_1), 3), '±', round( np.std(dice_list_1), 3))
print('Jaccard   : ', round(np.mean(jaccard_list_1), 3), '±', round( np.std(jaccard_list_1), 3))

Recall    :  0.937 ± 0.062
Precision :  0.932 ± 0.094
Accuracy  :  0.994 ± 0.005
DSC       :  0.929 ± 0.057
Jaccard   :  0.873 ± 0.091


In [29]:
recall_list = []
pricision_list = []
acc_list = []
dice_list = []
jaccard_list = []


for i in range(validation_label.shape[0]):
    
    seg1 = a2c_a4c_pred_4[i,:,:,0,1]
    gt1 = validation_label[i,:,:,0,1]

    seg1_n = seg1 == 0
    seg1_t = seg1 == 1

    gt1_n = gt1 == 0
    gt1_t = gt1 == 1


    tp = np.sum(seg1_t&gt1_t)

    fp = np.sum(seg1_t&gt1_n)

    tn = np.sum(seg1_n&gt1_n)

    fn = np.sum(gt1_t&seg1_n)


    recall = tp / (tp + fn)
    recall_list.append(recall)

#     print('Sensitivity : ', sensitivity)


    pricision = tp / (tp+fp)
    pricision_list.append(pricision)

#     print('Specificity : ', specificity)


    acc = (tp + tn) / (tp + tn + fp + fn)
    acc_list.append(acc)
#     print('Accuracy : ', acc)

    dice = (2*tp) / (2*tp + fp + fn)
    dice_list.append(dice)
#     print('DSC : ', dice)
    
    jaccard_list.append(jaccard_score(gt1, seg1, average='micro'))
    
    
dice_list_1 = np.array(dice_list)
recall_list_1 = np.array(recall_list)
pricision_list_1 = np.array(pricision_list)
jaccard_list_1 = np.array(jaccard_list)

dice_list_1 = dice_list_1[~np.isnan(dice_list_1)]
recall_list_1 = recall_list_1[~np.isnan(recall_list_1)]
pricision_list_1 = pricision_list_1[~np.isnan(pricision_list_1)]
jaccard_list_1 = jaccard_list_1[~np.isnan(jaccard_list_1)]

print('Recall    : ', round(np.mean(recall_list_1), 3), '±', round(np.std(recall_list_1), 3))
print('Precision : ', round( np.mean(pricision_list_1), 3), '±', round( np.std(pricision_list_1), 3))
print('Accuracy  : ', round(np.mean(acc_list), 3), '±', round( np.std(acc_list), 3))
print('DSC       : ', round(np.mean(dice_list_1), 3), '±', round( np.std(dice_list_1), 3))
print('Jaccard   : ', round(np.mean(jaccard_list_1), 3), '±', round( np.std(jaccard_list_1), 3))

Recall    :  0.95 ± 0.045
Precision :  0.962 ± 0.042
Accuracy  :  0.996 ± 0.003
DSC       :  0.955 ± 0.031
Jaccard   :  0.915 ± 0.052


In [ ]:
for sl in range(20):

    plt.figure(figsize=(20,20))

    plt.subplot(2,3,1)
    plt.imshow(test_image[sl,:,:,:,0], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,2)
    plt.imshow(test_label[sl,:,:,:,0], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,3)
    plt.imshow(test_image[sl,:,:,:,0], cmap='gray')
    plt.imshow(test_label[sl,:,:,:,0], cmap='Reds', alpha=0.25)
    plt.axis('off')

    plt.subplot(2,3,4)
    plt.imshow(test_image[sl,:,:,:,0], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,5)
    plt.imshow(a2c_a4c_pred_4[sl,:,:,:,0], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,6)
    plt.imshow(test_image[sl,:,:,:,0], cmap='gray')
    plt.imshow(a2c_a4c_pred_4[sl,:,:,:,0], cmap='Reds', alpha=0.25)
    plt.axis('off')


In [ ]:
for sl in range(20):

    plt.figure(figsize=(20,20))

    plt.subplot(2,3,1)
    plt.imshow(test_image[sl,:,:,:,1], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,2)
    plt.imshow(test_label[sl,:,:,:,1], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,3)
    plt.imshow(test_image[sl,:,:,:,1], cmap='gray')
    plt.imshow(test_label[sl,:,:,:,1], cmap='Reds', alpha=0.25)
    plt.axis('off')

    plt.subplot(2,3,4)
    plt.imshow(test_image[sl,:,:,:,1], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,5)
    plt.imshow(a2c_a4c_pred_4[sl,:,:,:,1], cmap='gray')
    plt.axis('off')

    plt.subplot(2,3,6)
    plt.imshow(test_image[sl,:,:,:,1], cmap='gray')
    plt.imshow(a2c_a4c_pred_4[sl,:,:,:,1], cmap='Reds', alpha=0.25)
    plt.axis('off')
